In [156]:
import os
# disable 

# import load_model
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib
matplotlib.use("Qt5Agg")

from enum import Enum
from tqdm import tqdm


In [190]:
prefix = './Results/progress'
log_path = '2023-01-15_17:46:04'

model_iteration = 'final'
model_path = os.path.join(prefix, log_path, 'models', f'model_{model_iteration}.h5')
history_path = os.path.join(prefix, log_path, 'reward_history.csv')
image_path = os.path.join(prefix, log_path, 'images')

### Load functions

In [171]:
original_items = ['event_time','original_open', 'original_close', 'original_high', 'original_low', 'original_volume']

class ActionSpace(Enum):
    BUY = 0
    HOLD = 1
    SELL = 2

def plot_data(data, ax):
    if ax is None:
        ax = plt.gca()
    ax.plot(data)
    # add trend line
    ax.plot(np.poly1d(np.polyfit(range(len(data)), data, 2))(range(len(data))))
    ax.set_xlabel("Episode")
    ax.legend(["Reward"])
    reward_save_path = os.path.join(image_path, f'average_reward_{model_iteration}.png')
    plt.savefig(reward_save_path)

def sequence_generator(data, batch_size, max_steps):
    for i in range(batch_size + 1, max_steps):
        yield data[i - batch_size:i]

def get_train_env_data(data):
    item = pd.DataFrame(data)
    return item[original_items], item.drop(original_items, axis=1)

def plot_buy_sell(data, ax):
    if ax is None:
        ax = plt.gca()

    width = .4
    width2 = .04

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i in range(0, len(data)):
        item = data[i]
        item_candlestick = item[0]
        action = item[1]
        index = item[2]



        if item_candlestick['original_close'] >= item_candlestick['original_open']:
            height = item_candlestick['original_close'] - item_candlestick['original_open']
            height_max = item_candlestick['original_high'] - item_candlestick['original_low']
            ax.bar(index, height, width, item_candlestick['original_open'], color=col1)
            ax.bar(index, height_max, width2, item_candlestick['original_open'], color=col1)
        else:
            height = item_candlestick['original_open'] - item_candlestick['original_close']
            height_max = item_candlestick['original_high'] - item_candlestick['original_low']
            ax.bar(index, height, width,  item_candlestick['original_close'], color=col2)
            ax.bar(index, height_max, width2,  item_candlestick['original_close'], color=col2)

        if action == ActionSpace.BUY:
            ax.scatter(index, item_candlestick['original_close'], marker='o', color='yellow', s=5)
        elif action == ActionSpace.SELL:
            ax.scatter(index, item_candlestick['original_close'], marker='o', color='blue', s=4)

    
    ax.set_ylabel("Price")

    ax.set_xlabel("Minutes")
    ax.legend()
    prediction_path = os.path.join(image_path, f'prediction_{model_iteration}.png')
    plt.savefig(prediction_path)


In [191]:
# load AI model from Models
model = load_model(model_path)
training_history = pd.read_csv(history_path)


# read data from file into pandas dataframe
df = pd.read_csv("./Data/dataset/VETUSDT.csv")


# loop through all sequences and predict buy hold or sell and put the results on a graph
buy_sells = []

LOOKBACK_WINDOW = 100
ITEMS = 500


seq_gen = sequence_generator(df, LOOKBACK_WINDOW, ITEMS)

for i in tqdm(range(0, ITEMS - LOOKBACK_WINDOW)):
    try:
        item = next(seq_gen)
    except:
        break
    train, env = get_train_env_data(item)
    env = env.to_numpy()
    prediction = model.predict(env.reshape(1, LOOKBACK_WINDOW, 17), verbose=0)
    action = ActionSpace(np.argmax(prediction))
    buy_sells.append((train.iloc[-1], action, i))

100%|█████████▉| 399/400 [00:17<00:00, 22.35it/s]


In [192]:

fig, ax = plt.subplots(nrows=2, ncols=1)
fig.tight_layout(pad=3)

plot_data(training_history['reward'], ax[0])
plot_buy_sell(buy_sells, ax[1])

plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


KeyboardInterrupt: 